**Includes**


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random as r
from sklearn.model_selection import train_test_split

**Importing Data**

In [0]:
X = pd.read_csv("train_t0.csv")
Y = pd.read_csv("train_t1.csv")
test = pd.read_csv("test_t0.csv")
del X['ID'], Y['ID'], test['ID']

**Remove Empty Columns**

In [0]:
X = X.loc[:, (X != 0).any(axis=0)]
test = test.loc[:, (test != 0).any(axis=0)]

**Remove Duplicate Columns**

In [0]:
column_count = len(X.columns)
dp_indexes = []
for i in range(0, column_count - 1):
  a = X.iloc[:, i]
  for j in range(i + 1, column_count):
    b = X.iloc[:, j]
    if np.array_equal(a, b):
      dp_indexes.append(j)
dp_indexes = list(set(dp_indexes))
X.drop(X.columns[dp_indexes], axis=1, inplace=True) 
test.drop(test.columns[dp_indexes], axis=1, inplace=True) 

**Applying PCA**

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 11)

X = pca.fit_transform(X)
test = pca.transform(test)

X = pd.DataFrame.from_records(X)
test = pd.DataFrame.from_records(test)

**5-Fold Cross Validation Using SVR**

In [0]:
import random as r
r.seed(1)

actual = []
predicted = []

from sklearn.svm import SVR
regressor = SVR(kernel='rbf', C=0.032, gamma='scale', epsilon=.022, tol=0.001)

from sklearn.model_selection import KFold
kf = KFold(n_splits=10, random_state=r.randint(0, 2147483648), shuffle=True)
for train_index, test_index in kf.split(X):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

  data = {} 
  dataframe = pd.DataFrame(data) 
  for i in range(0,595):
    regressor.fit(X_train, Y_train.iloc[:, i])
    pred = regressor.predict(X_test)
    dataframe.insert(i, i, pred, True)

  predicted.extend(dataframe.values)
  actual.extend(Y_test.values)

**Displaying MSE**

In [0]:
actualDF = pd.DataFrame.from_records(actual)
predDF = pd.DataFrame.from_records(predicted)

from sklearn.metrics import mean_squared_error as mse
actual = actualDF.to_numpy().flatten()
predicted = predDF.to_numpy().flatten()
mse(predicted,actual)

0.004084018606314645